In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *
from arcpy.ga import *
from arcpy.management import Delete
import geopandas as gpd
import pandas as pd
import os
import time
import numpy as np
from sklearn.neighbors import KDTree
import gc
from shapely.geometry import box
import shutil
import random
import matplotlib.pyplot as plt
import seaborn as sns
print('started')

#""" the HCOND1 values in the water wells represent the 
#lithology of the screen intervals where the HCOND2 values represent the saturated thickness in 
#the water wells (or the saturated thickness of the glacial aquifer where the aquifer thickness is 
#small). """"

arcpy.env.overwriteOutput = True
def calculate_rmse(predicted_values, known_values):
    """Calculate the root mean square error (RMSE) between predicted and known values."""
    predicted_values = np.array(predicted_values)
    known_values = np.array(known_values)
    square_errors = (predicted_values - known_values) ** 2
    mean_square_error = np.mean(square_errors)
    rmse = np.sqrt(mean_square_error)
    return rmse

def plot_krigging_vs_data(shape_temp, parameter, COUNTY):
    save_path = dic + fr'Fitness/{parameter}_{COUNTY}.png'
    filtered_data = shape_temp[(shape_temp.RASTERVALU > 0) & (shape_temp.COUNTY == COUNTY)]
    joint = sns.jointplot(x=filtered_data[parameter], y=filtered_data['RASTERVALU'], 
                          kind='scatter', joint_kws={"s": 5})
    joint.ax_joint.set_xlabel(f'Original {parameter}')
    joint.ax_joint.set_ylabel(f'Predicted {parameter}')
    joint.fig.suptitle(f'{COUNTY}')
    joint.ax_joint.grid(True, alpha=0.5)
    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.show()

    
def create_mask(cs, COUNTY, buffer_distance, michigan_boundary_path):
    mask = cs[cs.COUNTY == COUNTY].buffer(buffer_distance)
    mask_extent = mask.geometry.total_bounds
    shapefile_gdf = gpd.read_file(michigan_boundary_path)
    mask_polygon = box(*mask_extent)
    is_mask_within_shapefile = shapefile_gdf.geometry.contains(mask_polygon).any()
    
    if not is_mask_within_shapefile:
        mask = gpd.clip(mask, shapefile_gdf)

    return mask, mask_extent

dic= r"D:/GW_data_interpolated/Well_data_krigging/"  #### base directory
#,,'SWL','H_COND_2','V_COND_1', 'V_COND_2', 'TRANSMSV_1', 
parameters = ['TRANSMSV_2','TRANSMSV_1','AQ_THK_1', 'AQ_THK_2' ,'SWL', 'V_COND_1', 'V_COND_2','H_COND_2','H_COND_1']       
#groundwater_data_path=dic+r"WaterWells_Michigan_combined_projected_gr.pk1"

groundwater_data_path='D:/GW_data_interpolated/Grid/grid_points_well_obs_with_geometry.pk1'

counties_data_path=dic+'Counties_dis_gr.pk1'
michigan_boundary_path=dic+r"MichiganBoundry_26990.shp"
crs_setting='EPSG:26990'
#### cleaning and cover range
maximum_observed_head_allowed=520
buffer_distance_for_county = 5000

##### krigging parameters for optimization

STK=False
EBK=True

if EBK==True:
    print('EBK algorithm will be used')
    max_local_points_values = range(50, 256)
    overlap_factor_values = [x * 0.1 for x in range(10, 30)]  # this will generate values from 1.0 to 3.0 with a step of 0.1
    number_semivariograms_values = [50, 100, 200, 300]
    cell_size_values = [250]
    transformation_type_values = ["EMPIRICAL"]
    semivariogram_model_type_values = ["EXPONENTIAL", "EXPONENTIAL_DETRENDED", "WHITTLE", "WHITTLE_DETRENDED", "K_BESSEL", 'K_BESSEL_DETRENDED']

elif STK==True:
    print('STK algorithm will be used')
    semivariogram_model_type_values = ["SPHERICAL", "CIRCULAR", "EXPONENTIAL", "GAUSSIAN", "LINEAR"]
    cell_size_values = [250]
    nugget_values = [0, 0.5, 1.0]
    range_values = [100, 200, 300]
    lag_size_values = [5, 10, 15]
    major_range_values = [50, 100, 150]
    partial_sill_values = [10, 20, 30]
    lag_number_values = [5, 10, 15]

n_iterations = 10
up_threshorld=0.975
low_threshold=0.025

for parameter in parameters:
    GWD = gpd.GeoDataFrame(pd.read_pickle(groundwater_data_path), crs=crs_setting, geometry='geometry')
    cs = gpd.GeoDataFrame(pd.read_pickle(counties_data_path), crs=crs_setting, geometry='geometry')
    COUNTIES=sorted(GWD[~GWD.COUNTY.isna()].COUNTY.unique(), reverse=True)
    
    for COUNTY in COUNTIES:
        print(parameter, COUNTY)
        mask, mask_extent = create_mask(cs, COUNTY, buffer_distance_for_county, michigan_boundary_path)
        arcpy.env.extent = arcpy.Extent(mask_extent[0], mask_extent[1], mask_extent[2], mask_extent[3])
        path = dic+fr'Krigging_work_space/{parameter}_{COUNTY}'
        GWD_cleaned=GWD[~GWD[parameter].isna()][['COUNTY',parameter,'geometry']].reset_index(drop=True)
        GWD_cleaned=GWD_cleaned.clip(mask).reset_index(drop=True)
        GWD_cleaned = GWD_cleaned.drop_duplicates(subset='geometry').reset_index(drop=True)
        GWD_cleaned = GWD_cleaned[(GWD_cleaned[parameter] < GWD_cleaned[parameter].quantile(up_threshorld)) & (GWD_cleaned[parameter] > GWD_cleaned[parameter].quantile(low_threshold))].reset_index(drop=True)
        #GWD_cleaned = GWD_cleaned[GWD_cleaned.SWL<maximum_observed_head_allowed].reset_index(drop=True)
       
        GWD_cleaned.to_file(path)
        env.workspace = dic+r'Krigging_work_space'
        in_features = f"{parameter}_{COUNTY}/{parameter}_{COUNTY}.shp"
        z_field = parameter
        out_surface_raster = f"kriging_output_{parameter}_{COUNTY}.tif" 
        arcpy.env.overwriteOutput = True
        # Initialize variables for the best RMSE and parameters
        best_rmse = float('inf')
        best_params = None

        for _ in range(n_iterations):
            # Randomly select parameter values
            if EBK==True:
                cell_size = random.choice(cell_size_values)
                transformation_type = random.choice(transformation_type_values)
                max_local_points = random.choice(max_local_points_values)
                overlap_factor = random.choice(overlap_factor_values)
                number_semivariograms = random.choice(number_semivariograms_values)
                semivariogram_model_type = random.choice(semivariogram_model_type_values)
                
            elif STK==True:
                cell_size = random.choice(cell_size_values)
                semivariogram_model_type = random.choice(semivariogram_model_type_values)
                nugget = random.choice(nugget_values)
                range_value = random.choice(range_values)
                lag_size = random.choice(lag_size_values)
                major_range = random.choice(major_range_values)
                partial_sill = random.choice(partial_sill_values)
                lag_number = random.choice(lag_number_values)
                
            out_ga_layer = ""
            out_raster = out_surface_raster
            output_type_values = [ "PREDICTION_STANDARD_ERROR","PREDICTION"]
    
            for output_type in output_type_values:
                if output_type == "PREDICTION_STANDARD_ERROR":
                    out_raster_error = f"kriging_stderr_{parameter}_{COUNTY}.tif" 
                else:
                    out_raster_pred = f"kriging_output_{parameter}_{COUNTY}.tif" 
                    out_raster_best_pred = f"kriging_best_{parameter}_{COUNTY}.tif" 

            # Execute Empirical Bayesian Kriging with current parameters
            
            if EBK==True:
                arcpy.EmpiricalBayesianKriging_ga(in_features, z_field, out_ga_layer, out_raster_pred, cell_size, 
                                                  transformation_type, max_local_points, overlap_factor, 
                                                  number_semivariograms, "", 'PREDICTION', "", "", "", 
                                                  semivariogram_model_type)
            elif STK==True:
                kriging_model = KrigingModelOrdinary(semivariogram_model_type, lag_size, major_range, partial_sill, nugget)
                outKriging = Kriging(in_features, z_field, kriging_model, cell_size, "VARIABLE", None)
                outKriging.save(out_raster_pred)


            tif_path = dic+fr'Krigging_work_space/kriging_output_{parameter}_{COUNTY}.tif'
            shapefile_path = dic+fr'Krigging_work_space/{parameter}_{COUNTY}/{parameter}_{COUNTY}.shp'
            arcpy.env.workspace = dic+fr'Krigging_work_space/'
            arcpy.sa.ExtractValuesToPoints(shapefile_path, tif_path, "calibration_temp.shp", "INTERPOLATE", "VALUE_ONLY")
            shape_temp=gpd.read_file(dic+"Krigging_work_space/calibration_temp.shp")
            rmse=calculate_rmse(shape_temp[(shape_temp.RASTERVALU>0) & (shape_temp.COUNTY==COUNTY)][parameter].values, shape_temp[(shape_temp.RASTERVALU>0) & (shape_temp.COUNTY==COUNTY)].RASTERVALU.values )    
            print('RMSE: ',rmse)
            if rmse < best_rmse:
                print('better solution foound')
                if EBK==True:
                    arcpy.EmpiricalBayesianKriging_ga(in_features, z_field, out_ga_layer, out_raster_error, cell_size, 
                                      transformation_type, max_local_points, overlap_factor, 
                                      number_semivariograms, "", "PREDICTION_STANDARD_ERROR", "", "", "", 
                                      semivariogram_model_type)

                    arcpy.EmpiricalBayesianKriging_ga(in_features, z_field, out_ga_layer, out_raster_best_pred, cell_size, 
                                      transformation_type, max_local_points, overlap_factor, 
                                      number_semivariograms, "", 'PREDICTION', "", "", "", 
                                      semivariogram_model_type)  
                    
                    
                best_rmse = rmse
                plot_krigging_vs_data(shape_temp, parameter, COUNTY)
            time.sleep(1)
            gc.collect()
        directory_path = dic+'Krigging_work_space'
        for item in os.listdir(dic+'Krigging_work_space'):
            item_path = os.path.join(directory_path, item)
            if os.path.isdir(item_path) and item != 'clip':
                shutil.rmtree(item_path)
                

In [ ]:
import glob
import os
import arcpy
from arcpy import env
from arcpy.sa import *
import geopandas as gpd
import pandas as pd
arcpy.env.overwriteOutput = True
import shutil

def masking_krigged_data(cs,input_rasters):
    for raster in input_rasters:
        # Extract county name from the raster file name
        county_name = raster.split('\\')[-1].split(f'kriging_{typ}_')[1].split('.')[0].split(f'{parameter}_')[1]
        # Get the polygon corresponding to the county
        county_polygon = cs[cs.COUNTY==county_name].buffer(500)
        # Save the county polygon to a temporary shapefile
        county_polygon.to_file(county_name + '_temp.shp')
        # Define the output raster file name
        output_raster = raster.replace('.tif', '_masked.tif')
        # Extract by mask
        outExtractByMask = ExtractByMask(raster, county_name + '_temp.shp')
        # Save the output 
        outExtractByMask.save(output_raster)

def creating_mosaic_from_krigged_masked(cs,input_rasters,out_dic):
    output_location = dic+'clip'
    arcpy.env.overwriteOutput = True
    input_rasters = glob.glob(dic+fr'/kriging_{typ}_{parameter}_*masked.tif')
    mosaic = arcpy.MosaicToNewRaster_management(input_rasters, output_location, f'{parameter}_{typ}_mosaic.tif', '', '64_BIT', '250', '1', 'BLEND')
    mask_geometry = cs.geometry.unary_union
    # Save the mask as a shapefile
    mask_shp = gpd.GeoDataFrame(gpd.GeoSeries(mask_geometry), columns=['geometry'])
    mask_shp.set_crs(epsg=26990, inplace=True)
    mask_shp.to_file('mask.shp')
    # Now use 'mask.shp' as the mask in the ExtractByMask function
    outExtractByMask = ExtractByMask(mosaic, 'mask.shp')
    output_raster = os.path.join(out_dic, f'{typ}_{parameter}.tif')
    # Save the output 
    outExtractByMask.save(output_raster)

typs=['best','stderr']
dic=r'C:/Users/rafieiva/OneDrive - Michigan State University/Desktop/GW_data_interpolated/Well_data_krigging/Krigging_work_space/'
out_dic=r'C:/Users/rafieiva/OneDrive - Michigan State University/Desktop/GW_data_interpolated/Well_data_krigging/Krigged_data/'
parameters=['SWL', 'AQ_THK_1','AQ_THK_2','V_COND_2', 'V_COND_1','TRANSMSV_1','TRANSMSV_2','H_COND_1','H_COND_2']
arcpy.env.workspace = dic

for parameter in parameters:
    for typ in typs:
        file_list = glob.glob(os.path.join( dic+'clip', '*'))
        for file_path in file_list:
            os.remove(file_path)
        print('Files in clip folder are deleted ')
        mask_removal = glob.glob(dic+'*masked.tif')
        for file in mask_removal:
            os.remove(file)
        print(' masked files in input folder have been deleted')
        cs = gpd.GeoDataFrame(pd.read_pickle('Counties_dis_gr.pk1'), crs='EPSG:26990')
        folder_path = os.path.join(dic, 'clip')

        os.makedirs(folder_path, exist_ok=True)
        env.workspace = dic
        arcpy.env.nodata = 'NONE'
        input_rasters = glob.glob(dic+f'kriging_{typ}_{parameter}_*.tif')
        masking_krigged_data(cs,input_rasters)
        print('All rasters have been masked.')
        creating_mosaic_from_krigged_masked(cs,input_rasters,out_dic)

        if typ=='best':
            if parameter in ['V_COND_1','V_COND_2','H_COND_1','H_COND_2']:
                out_min = 0
                out_max = 300 
            elif parameter in ['TRANSMSV_1','TRANSMSV_2']:
                out_min = 0.0004
                out_max = 72100
            elif parameter in ['AQ_THK_1','AQ_THK_2']:
                out_min = 1
                out_max = 478.0       
            elif parameter in ['SWL']:
                out_min = 0
                out_max = 511
            
            temp_path=os.path.join(out_dic, fr'best_{parameter}.tif')
            saved_raster = arcpy.Raster(temp_path)
            neighborhood = NbrRectangle(3, 3, 'CELL') # You can adjust the size
            smoothed_raster = FocalStatistics(saved_raster, neighborhood, 'MEAN')
            temp_path=os.path.join(out_dic, fr'best_{parameter}_smoothed.tif')
            smoothed_raster.save(temp_path)
            print('smoothing compeleted')
            temp_path=os.path.join(out_dic,fr'best_{parameter}_smoothed.tif')
            smoothed_raster = arcpy.Raster(temp_path)
            in_min = smoothed_raster.minimum
            in_max = smoothed_raster.maximum
            scale = (out_max - out_min) / (in_max - in_min)
            base = out_min - in_min * scale
            scaled_raster = smoothed_raster * scale + base
            temp_path=os.path.join(out_dic, fr'best_{parameter}_smoothed_rescaled.tif')
            scaled_raster.save(temp_path)
            print('rescaling compeleted')
        else:
            temp_path=os.path.join(out_dic, fr'stderr_{parameter}.tif')
            saved_raster = arcpy.Raster(temp_path)
            neighborhood = NbrRectangle(3, 3, 'CELL') # You can adjust the size
            smoothed_raster = FocalStatistics(saved_raster, neighborhood, 'MEAN')
            # Save the output
            temp_path=os.path.join(out_dic, fr'stderr_{parameter}_smoothed.tif')
            smoothed_raster.save(temp_path)
            print('smoothing compeleted')